In [10]:
# 📚 Imports et Configuration Complète
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple, Any
import warnings
warnings.filterwarnings('ignore')

# Configuration matplotlib pour de beaux graphiques
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 12
sns.set_palette("husl")

# 🔧 Configuration des chemins - Ajouter le projet et les modules
project_root = os.path.abspath('../../')
sys.path.insert(0, project_root)
sys.path.insert(0, os.path.join(project_root, 'src'))
sys.path.insert(0, os.path.join(project_root, 'game', 'secret_env'))

print("🚀 INITIALISATION DE L'ANALYSE MONTE CARLO")
print("=" * 60)

# 🎮 Import des environnements secrets
try:
    from secret_envs_wrapper import SecretEnv0, SecretEnv1, SecretEnv2, SecretEnv3
    print("✅ Environnements secrets importés avec succès")
    
    # Test rapide des environnements
    env_configs = [
        ("SecretEnv0", SecretEnv0),
        ("SecretEnv1", SecretEnv1), 
        ("SecretEnv2", SecretEnv2),
        ("SecretEnv3", SecretEnv3)
    ]
    
    env_info = {}
    for env_name, env_class in env_configs:
        try:
            env = env_class()
            states = env.num_states()
            actions = env.num_actions()
            env_info[env_name] = {'states': states, 'actions': actions, 'class': env_class}
            print(f"📊 {env_name} - États: {states}, Actions: {actions}")
        except Exception as e:
            print(f"❌ Erreur avec {env_name}: {e}")
            env_info[env_name] = None
    
    print(f"\n🎉 {len([k for k,v in env_info.items() if v is not None])} environnements secrets fonctionnels !")
    
except Exception as e:
    print(f"❌ Erreur d'import des environnements secrets: {e}")
    print("Vérifiez que les bibliothèques natives sont présentes dans ./libs/")
    env_info = {}

# 🧠 Import des algorithmes Monte Carlo existants du projet
try:
    from monte_carlo import MonteCarloES, OnPolicyMC, OffPolicyMC
    print("✅ Algorithmes Monte Carlo importés depuis src/monte_carlo.py")
    print("   • MonteCarloES - Monte Carlo avec Exploring Starts")
    print("   • OnPolicyMC - On-policy First-Visit Monte Carlo")  
    print("   • OffPolicyMC - Off-policy Monte Carlo avec Importance Sampling")
    
    monte_carlo_available = True
    
except Exception as e:
    print(f"❌ Erreur d'import des algorithmes Monte Carlo: {e}")
    print("Vérifiez que le module src/monte_carlo.py est accessible")
    monte_carlo_available = False

print("\n🔧 Configuration terminée !")
print("=" * 60)


🚀 INITIALISATION DE L'ANALYSE MONTE CARLO
✅ Environnements secrets importés avec succès
📊 SecretEnv0 - États: 8192, Actions: 3
📊 SecretEnv1 - États: 65536, Actions: 3
📊 SecretEnv2 - États: 2097152, Actions: 3
📊 SecretEnv3 - États: 65536, Actions: 3

🎉 4 environnements secrets fonctionnels !
✅ Algorithmes Monte Carlo importés depuis src/monte_carlo.py
   • MonteCarloES - Monte Carlo avec Exploring Starts
   • OnPolicyMC - On-policy First-Visit Monte Carlo
   • OffPolicyMC - Off-policy Monte Carlo avec Importance Sampling

🔧 Configuration terminée !


In [11]:
# 🔧 Adaptateur pour les Algorithmes Monte Carlo Existants

class SecretEnvMCAdapter:
    """
    Adaptateur spécialement conçu pour les environnements secrets afin qu'ils soient 
    compatibles avec les algorithmes Monte Carlo existants (MonteCarloES, OnPolicyMC, OffPolicyMC).
    
    Implémente l'interface attendue par src/monte_carlo.py
    """
    
    def __init__(self, secret_env_class, env_name="SecretEnv"):
        self.secret_env_class = secret_env_class
        self.env_name = env_name
        
        # Obtenir les propriétés MDP depuis une instance temporaire
        temp_env = secret_env_class()
        self.nS = temp_env.num_states()  # Propriété attendue par Monte Carlo
        self.nA = temp_env.num_actions() # Propriété attendue par Monte Carlo
        
        # État courant et environnement
        self.current_env = None
        self.current_state = None
        self.last_score = 0.0
        self.episode_steps = 0
        
        print(f"🏗️  {env_name} MC-Adapter - États: {self.nS}, Actions: {self.nA}")
    
    def reset(self):
        """
        Réinitialise l'environnement - Interface Gym standard
        Retourne l'état initial (int pour l'environnement discret)
        """
        try:
            # Créer une nouvelle instance pour chaque épisode
            self.current_env = self.secret_env_class()
            self.current_env.reset()
            
            # Obtenir l'état initial
            self.current_state = self.current_env.state_id()
            self.last_score = self.current_env.score()
            self.episode_steps = 0
            
            return self.current_state  # Retourner directement l'état (pas de tuple)
            
        except Exception as e:
            print(f"❌ Erreur reset {self.env_name}: {e}")
            # Retourner un état par défaut en cas d'erreur
            return 0
    
    def step(self, action):
        """
        Exécute une action - Interface Gym standard
        Retourne (next_state, reward, done, info)
        """
        try:
            if self.current_env is None:
                # Réinitialiser si pas d'environnement
                self.reset()
            
            # Vérifier si l'action est valide dans cet environnement
            available_actions = self._get_available_actions()
            if action not in available_actions:
                # Action non valide - petite pénalité mais continuer
                return self.current_state, -0.05, False, {'invalid_action': True}
            
            # Sauvegarder score avant action
            old_score = self.current_env.score()
            
            # Exécuter l'action
            self.current_env.step(action)
            self.episode_steps += 1
            
            # Calculer les résultats
            next_state = self.current_env.state_id()
            new_score = self.current_env.score()
            reward = new_score - old_score  # Récompense différentielle
            done = self.current_env.is_game_over()
            
            # Mettre à jour l'état
            self.current_state = next_state
            self.last_score = new_score
            
            # Informations supplémentaires
            info = {
                'available_actions': available_actions,
                'cumulative_score': new_score,
                'episode_steps': self.episode_steps
            }
            
            # Limite de sécurité pour éviter les épisodes infinis
            if self.episode_steps > 500:  # Limite plus stricte pour les tests
                done = True
                reward -= 0.5  # Légère pénalité pour épisodes trop longs
                info['timeout'] = True
            
            return next_state, reward, done, info
            
        except Exception as e:
            # En cas d'erreur, terminer l'épisode avec pénalité
            return self.current_state, -1.0, True, {'error': str(e)}
    
    def _get_available_actions(self):
        """Obtient les actions disponibles dans l'état courant"""
        try:
            if self.current_env is None:
                return list(range(self.nA))
            
            actions = self.current_env.available_actions()
            return list(actions) if len(actions) > 0 else list(range(self.nA))
        except:
            # Fallback : toutes les actions disponibles
            return list(range(self.nA))
    
    def get_mdp_info(self):
        """Informations MDP pour compatibilité - utilisé par certains algorithmes"""
        return {
            'states': list(range(self.nS)),
            'actions': list(range(self.nA)),
            'n_states': self.nS,
            'n_actions': self.nA,
            'name': self.env_name
        }

# 🏗️ Création des adaptateurs pour les environnements secrets
print("\n🏗️ CRÉATION DES ADAPTATEURS MONTE CARLO")
print("-" * 50)

adapters = {}
successful_adapters = 0

if env_info:  # Si les environnements sont disponibles
    for env_name, env_data in env_info.items():
        if env_data is not None:  # Si l'environnement est fonctionnel
            try:
                adapter = SecretEnvMCAdapter(env_data['class'], env_name)
                
                # Test rapide de l'adaptateur
                test_state = adapter.reset()
                test_next_state, test_reward, test_done, test_info = adapter.step(0)
                
                adapters[env_name] = adapter
                successful_adapters += 1
                
                print(f"✅ {env_name}: Adapter créé et testé (état initial: {test_state})")
                
            except Exception as e:
                print(f"❌ {env_name}: Erreur creation adapter - {e}")
                adapters[env_name] = None
        else:
            print(f"❌ {env_name}: Environnement non disponible")
            adapters[env_name] = None

    print(f"\n🎉 {successful_adapters}/{len(env_info)} adaptateurs Monte Carlo créés avec succès !")
else:
    print("❌ Aucun environnement secret disponible")

if successful_adapters == 0:
    print("⚠️ Aucun adaptateur fonctionnel - L'analyse ne pourra pas continuer")

print("-" * 50)



🏗️ CRÉATION DES ADAPTATEURS MONTE CARLO
--------------------------------------------------
🏗️  SecretEnv0 MC-Adapter - États: 8192, Actions: 3
✅ SecretEnv0: Adapter créé et testé (état initial: 0)
🏗️  SecretEnv1 MC-Adapter - États: 65536, Actions: 3
✅ SecretEnv1: Adapter créé et testé (état initial: 0)
🏗️  SecretEnv2 MC-Adapter - États: 2097152, Actions: 3
✅ SecretEnv2: Adapter créé et testé (état initial: 0)
🏗️  SecretEnv3 MC-Adapter - États: 65536, Actions: 3
✅ SecretEnv3: Adapter créé et testé (état initial: 0)

🎉 4/4 adaptateurs Monte Carlo créés avec succès !
--------------------------------------------------


In [12]:
# 🧠 Entraînement avec les Algorithmes Monte Carlo Existants

def run_monte_carlo_analysis(num_episodes=300, gamma=0.99):
    """
    Lance l'analyse complète en utilisant les algorithmes Monte Carlo existants du projet.
    
    Args:
        num_episodes: Nombre d'épisodes d'entraînement par algorithme
        gamma: Facteur de discount pour tous les algorithmes
    
    Returns:
        dict: Résultats complets de l'analyse
    """
    
    print("\n🚀 LANCEMENT DE L'ANALYSE MONTE CARLO")
    print("=" * 60)
    print(f"⚙️  Paramètres: {num_episodes} épisodes, γ={gamma}")
    print(f"🎮 Environnements à tester: {len([k for k,v in adapters.items() if v is not None])}")
    print(f"🧠 Algorithmes: MonteCarloES, OnPolicyMC, OffPolicyMC")
    print("=" * 60)
    
    if not monte_carlo_available:
        print("❌ Algorithmes Monte Carlo non disponibles !")
        return {}
    
    all_results = {}
    total_combinations = 0
    completed_combinations = 0
    
    # Compter le total de combinaisons
    for env_name, adapter in adapters.items():
        if adapter is not None:
            total_combinations += 3  # 3 algorithmes par environnement
    
    # Entraînement pour chaque environnement
    for env_name, adapter in adapters.items():
        if adapter is None:
            print(f"⏭️  Skipping {env_name} (adapter non disponible)")
            continue
        
        print(f"\n🎮 ENVIRONNEMENT: {env_name}")
        print(f"   États: {adapter.nS}, Actions: {adapter.nA}")
        print("-" * 50)
        
        env_results = {}
        
        # 1. 🎯 Monte Carlo Exploring Starts
        print(f"\\n🔥 [1/3] Monte Carlo Exploring Starts...")
        try:
            mc_es = MonteCarloES(adapter, gamma=gamma)
            print(f"   🏗️  MonteCarloES initialisé pour {env_name}")
            
            result_es = mc_es.train(num_episodes=num_episodes)
            result_es['algorithm'] = 'MonteCarloES'
            result_es['env_name'] = env_name
            
            # Ajouter évaluation finale
            eval_results = mc_es.evaluate(num_episodes=50)
            result_es['evaluation'] = eval_results
            
            env_results['MonteCarloES'] = result_es
            completed_combinations += 1
            
            print(f"   ✅ MonteCarloES terminé - Récompense finale: {result_es['history'][-1]['reward']:.3f}")
            
        except Exception as e:
            print(f"   ❌ Erreur MonteCarloES: {e}")
            env_results['MonteCarloES'] = {'history': [], 'error': str(e)}
        
        # 2. 🎯 On-Policy Monte Carlo  
        print(f"\\n🔄 [2/3] On-Policy Monte Carlo...")
        try:
            on_policy_mc = OnPolicyMC(adapter, gamma=gamma, epsilon=0.3)
            print(f"   🏗️  OnPolicyMC initialisé pour {env_name} (ε=0.3)")
            
            result_on = on_policy_mc.train(num_episodes=num_episodes)
            result_on['algorithm'] = 'OnPolicyMC'
            result_on['env_name'] = env_name
            
            # Ajouter évaluation finale
            eval_results = on_policy_mc.evaluate(num_episodes=50)
            result_on['evaluation'] = eval_results
            
            env_results['OnPolicyMC'] = result_on
            completed_combinations += 1
            
            print(f"   ✅ OnPolicyMC terminé - Récompense finale: {result_on['history'][-1]['reward']:.3f}")
            
        except Exception as e:
            print(f"   ❌ Erreur OnPolicyMC: {e}")
            env_results['OnPolicyMC'] = {'history': [], 'error': str(e)}
        
        # 3. 🎯 Off-Policy Monte Carlo
        print(f"\\n⚖️  [3/3] Off-Policy Monte Carlo...")
        try:
            off_policy_mc = OffPolicyMC(adapter, gamma=gamma, epsilon=0.4)
            print(f"   🏗️  OffPolicyMC initialisé pour {env_name} (ε=0.4)")
            
            result_off = off_policy_mc.train(num_episodes=num_episodes)
            result_off['algorithm'] = 'OffPolicyMC'  
            result_off['env_name'] = env_name
            
            # Ajouter évaluation finale
            eval_results = off_policy_mc.evaluate(num_episodes=50)
            result_off['evaluation'] = eval_results
            
            env_results['OffPolicyMC'] = result_off
            completed_combinations += 1
            
            print(f"   ✅ OffPolicyMC terminé - Récompense finale: {result_off['history'][-1]['reward']:.3f}")
            
        except Exception as e:
            print(f"   ❌ Erreur OffPolicyMC: {e}")
            env_results['OffPolicyMC'] = {'history': [], 'error': str(e)}
        
        # Stocker les résultats pour cet environnement
        all_results[env_name] = env_results
        
        # Résumé pour cet environnement
        print(f"\\n📊 RÉSUMÉ {env_name}:")
        for alg_name, result in env_results.items():
            if 'history' in result and result['history']:
                final_reward = result['history'][-1]['reward']
                avg_reward = np.mean([h['reward'] for h in result['history'][-10:]])
                print(f"   • {alg_name}: Récompense finale = {final_reward:.3f}, Moyenne récente = {avg_reward:.3f}")
            else:
                print(f"   • {alg_name}: ❌ Échec")
    
    # Résumé global
    print(f"\\n🎉 ANALYSE COMPLÈTE TERMINÉE !")
    print(f"📈 {completed_combinations}/{total_combinations} combinaisons réussies")
    print("=" * 60)
    
    return all_results

# 🚀 Lancement de l'analyse complète
if successful_adapters > 0 and monte_carlo_available:
    print("⏳ Lancement de l'analyse (peut prendre 5-15 minutes selon les paramètres)...")
    
    # Paramètres d'entraînement - ajustables selon les besoins
    EPISODES = 400  # Nombre d'épisodes par algorithme (augmentez pour plus de précision)
    GAMMA = 0.99    # Facteur de discount
    
    all_results = run_monte_carlo_analysis(num_episodes=EPISODES, gamma=GAMMA)
    
else:
    print("❌ Impossible de lancer l'analyse :")
    if successful_adapters == 0:
        print("   - Aucun adaptateur d'environnement fonctionnel")
    if not monte_carlo_available:
        print("   - Algorithmes Monte Carlo non importés")
    
    all_results = {}


⏳ Lancement de l'analyse (peut prendre 5-15 minutes selon les paramètres)...

🚀 LANCEMENT DE L'ANALYSE MONTE CARLO
⚙️  Paramètres: 400 épisodes, γ=0.99
🎮 Environnements à tester: 4
🧠 Algorithmes: MonteCarloES, OnPolicyMC, OffPolicyMC

🎮 ENVIRONNEMENT: SecretEnv0
   États: 8192, Actions: 3
--------------------------------------------------
\n🔥 [1/3] Monte Carlo Exploring Starts...
   🏗️  MonteCarloES initialisé pour SecretEnv0


: 

In [ ]:
# 📊 Fonctions de Visualisation et d'Analyse

def plot_learning_curves(results_dict, title_prefix=""):
    """Affiche les courbes d'apprentissage pour tous les algorithmes d'un environnement"""
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    axes = axes.flatten()
    colors = ['#e74c3c', '#3498db', '#2ecc71', '#f39c12']  # Rouge, Bleu, Vert, Orange
    
    # 1. Récompenses par épisode avec moyenne mobile
    ax1 = axes[0]
    for i, (alg_name, result) in enumerate(results_dict.items()):
        if 'history' in result and result['history']:
            history = result['history']
            episodes = [h['episode'] for h in history]
            rewards = [h['reward'] for h in history]
            
            # Moyenne mobile pour lisser les courbes
            window_size = min(30, len(rewards) // 10 + 1)
            if len(rewards) >= window_size:
                rewards_smooth = pd.Series(rewards).rolling(window=window_size, min_periods=1).mean()
                ax1.plot(episodes, rewards_smooth, label=alg_name, color=colors[i % len(colors)], linewidth=2.5)
                ax1.plot(episodes, rewards, alpha=0.3, color=colors[i % len(colors)], linewidth=0.8)
            else:
                ax1.plot(episodes, rewards, label=alg_name, color=colors[i % len(colors)], linewidth=2)
    
    ax1.set_title(f'{title_prefix} - Récompenses par Épisode', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Épisode')
    ax1.set_ylabel('Récompense')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Q-values moyennes (si disponibles)
    ax2 = axes[1]
    for i, (alg_name, result) in enumerate(results_dict.items()):
        if 'history' in result and result['history']:
            history = result['history']
            episodes = [h['episode'] for h in history]
            
            # Vérifier si avg_q est disponible dans l'historique
            if 'avg_q' in history[0]:
                avg_q = [h['avg_q'] for h in history]
                ax2.plot(episodes, avg_q, label=alg_name, color=colors[i % len(colors)], linewidth=2)
    
    ax2.set_title(f'{title_prefix} - Évolution des Q-values Moyennes', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Épisode')
    ax2.set_ylabel('Q-value Moyenne')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. Longueur des épisodes
    ax3 = axes[2]
    for i, (alg_name, result) in enumerate(results_dict.items()):
        if 'history' in result and result['history']:
            history = result['history']
            episodes = [h['episode'] for h in history]
            
            # Vérifier le format des longueurs d'épisode
            if 'length' in history[0]:
                lengths = [h['length'] for h in history]
            elif 'episode_length' in history[0]:
                lengths = [h['episode_length'] for h in history]  
            else:
                continue
            
            # Moyenne mobile
            window_size = min(30, len(lengths) // 10 + 1)
            if len(lengths) >= window_size:
                lengths_smooth = pd.Series(lengths).rolling(window=window_size, min_periods=1).mean()
                ax3.plot(episodes, lengths_smooth, label=alg_name, color=colors[i % len(colors)], linewidth=2)
            else:
                ax3.plot(episodes, lengths, label=alg_name, color=colors[i % len(colors)], linewidth=2)
    
    ax3.set_title(f'{title_prefix} - Longueur des Épisodes', fontsize=14, fontweight='bold')
    ax3.set_xlabel('Épisode')
    ax3.set_ylabel('Nombre de Steps')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # 4. Convergence - Variation des récompenses (stabilité)
    ax4 = axes[3]
    for i, (alg_name, result) in enumerate(results_dict.items()):
        if 'history' in result and result['history']:
            history = result['history']
            episodes = []
            stds = []
            
            window_size = 50
            rewards = [h['reward'] for h in history]
            for j in range(window_size, len(history)):
                recent_rewards = rewards[j-window_size:j]
                episodes.append(history[j]['episode'])
                stds.append(np.std(recent_rewards))
            
            if len(episodes) > 0:
                ax4.plot(episodes, stds, label=alg_name, color=colors[i % len(colors)], linewidth=2)
    
    ax4.set_title(f'{title_prefix} - Stabilité (Écart-type des récompenses)', fontsize=14, fontweight='bold')
    ax4.set_xlabel('Épisode')
    ax4.set_ylabel('Écart-type des récompenses')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

def analyze_results_detailed(all_results):
    """Analyse détaillée avec métriques de performance"""
    print("🔍 ANALYSE DÉTAILLÉE DES RÉSULTATS")
    print("=" * 70)
    
    summary_data = []
    
    for env_name, env_results in all_results.items():
        print(f"\n📊 {env_name.upper()}")
        print("-" * 50)
        
        for alg_name, result in env_results.items():
            if 'history' in result and result['history']:
                history = result['history']
                
                # Métriques de base
                total_episodes = len(history)
                all_rewards = [h['reward'] for h in history]
                avg_reward = np.mean(all_rewards)
                std_reward = np.std(all_rewards)
                
                # Performance finale (derniers 20% d'épisodes)
                final_portion = history[int(0.8 * len(history)):]
                final_rewards = [h['reward'] for h in final_portion]
                final_avg_reward = np.mean(final_rewards) if final_rewards else 0
                final_stability = np.std(final_rewards) if len(final_rewards) > 1 else 0
                
                # Métriques d'évaluation si disponibles
                eval_info = ""
                if 'evaluation' in result:
                    eval_data = result['evaluation']
                    eval_reward = eval_data.get('avg_reward', eval_data.get('average_reward', 0))
                    success_rate = eval_data.get('success_rate', 0)
                    eval_info = f", Eval: {eval_reward:.3f} (Succès: {success_rate:.1%})"
                
                print(f"\n🎯 {alg_name}:")
                print(f"   • Récompense moyenne: {avg_reward:.3f} (±{std_reward:.3f})")
                print(f"   • Performance finale: {final_avg_reward:.3f}")
                print(f"   • Stabilité finale: {final_stability:.3f}")
                print(f"   • Épisodes total: {total_episodes}{eval_info}")
                
                # Caractéristiques spécifiques aux algorithmes
                if len(history) > 0:
                    if 'epsilon' in history[0] and 'epsilon' in history[-1]:
                        initial_eps = history[0]['epsilon']
                        final_eps = history[-1]['epsilon']
                        print(f"   • Décroissance ε: {initial_eps:.3f} → {final_eps:.3f}")
                
                # Ajouter aux données de résumé
                summary_data.append({
                    'Environnement': env_name,
                    'Algorithme': alg_name,
                    'Récompense_Moyenne': f"{avg_reward:.3f}",
                    'Récompense_Finale': f"{final_avg_reward:.3f}",
                    'Stabilité': f"{final_stability:.3f}",
                    'Épisodes': total_episodes
                })
                
            elif 'error' in result:
                print(f"\n❌ {alg_name}: Erreur - {result['error']}")
                summary_data.append({
                    'Environnement': env_name,
                    'Algorithme': alg_name,
                    'Récompense_Moyenne': "ERREUR",
                    'Récompense_Finale': "ERREUR",
                    'Stabilité': "N/A",
                    'Épisodes': 0
                })
            else:
                print(f"\n❌ {alg_name}: Aucune donnée valide")
                summary_data.append({
                    'Environnement': env_name,
                    'Algorithme': alg_name,
                    'Récompense_Moyenne': "0.000",
                    'Récompense_Finale': "0.000",
                    'Stabilité': "N/A",
                    'Épisodes': 0
                })
    
    # Tableau récapitulatif
    summary_df = pd.DataFrame(summary_data)
    print("\n📋 TABLEAU RÉCAPITULATIF COMPLET:")
    print("=" * 70)
    print(summary_df.to_string(index=False))
    
    return summary_df

def plot_performance_heatmap(all_results):
    """Heatmap des performances finales par algorithme et environnement"""
    
    # Préparer les données pour la heatmap
    env_names = list(all_results.keys())
    alg_names = ['MonteCarloES', 'OnPolicyMC', 'OffPolicyMC']
    
    # Matrice des performances
    performance_matrix = []
    
    for env_name in env_names:
        env_row = []
        for alg_name in alg_names:
            if env_name in all_results and alg_name in all_results[env_name]:
                result = all_results[env_name][alg_name]
                if 'history' in result and result['history']:
                    # Performance finale (moyenne des 20 derniers épisodes)
                    final_rewards = [h['reward'] for h in result['history'][-20:]]
                    final_performance = np.mean(final_rewards)
                else:
                    final_performance = 0.0
            else:
                final_performance = 0.0
            
            env_row.append(final_performance)
        performance_matrix.append(env_row)
    
    # Créer la heatmap
    plt.figure(figsize=(10, 8))
    heatmap = plt.imshow(performance_matrix, cmap='RdYlGn', aspect='auto')
    
    # Personnaliser la heatmap
    plt.xticks(range(len(alg_names)), alg_names, rotation=45)
    plt.yticks(range(len(env_names)), env_names)
    plt.xlabel('Algorithmes Monte Carlo')
    plt.ylabel('Environnements Secrets')
    plt.title('🌡️ Heatmap des Performances Finales\\n(Récompense moyenne des 20 derniers épisodes)', 
              fontsize=14, fontweight='bold')
    
    # Ajouter les valeurs dans les cellules
    for i in range(len(env_names)):
        for j in range(len(alg_names)):
            value = performance_matrix[i][j]
            color = 'white' if abs(value) > 0.5 else 'black'
            plt.text(j, i, f'{value:.3f}', ha='center', va='center', 
                    color=color, fontweight='bold', fontsize=11)
    
    plt.colorbar(heatmap, label='Performance Finale')
    plt.tight_layout()
    plt.show()

def generate_recommendations(all_results):
    """Génère des recommandations basées sur l'analyse"""
    
    print("💡 RECOMMANDATIONS ET CONCLUSIONS")
    print("=" * 70)
    
    # Trouver le meilleur algorithme par environnement
    best_performers = {}
    for env_name, env_results in all_results.items():
        best_alg = None
        best_score = -float('inf')
        
        for alg_name, result in env_results.items():
            if 'history' in result and result['history']:
                # Score composite : performance finale + stabilité
                final_rewards = [h['reward'] for h in result['history'][-20:]]
                if final_rewards:
                    avg_performance = np.mean(final_rewards)
                    stability = -np.std(final_rewards)  # Negative car moins de variance = mieux
                    composite_score = avg_performance * 0.8 + stability * 0.2
                    
                    if composite_score > best_score:
                        best_score = composite_score
                        best_alg = alg_name
        
        best_performers[env_name] = (best_alg, best_score)
    
    print("\n🏆 MEILLEURS ALGORITHMES PAR ENVIRONNEMENT:")
    for env_name, (best_alg, score) in best_performers.items():
        if best_alg:
            print(f"   • {env_name}: {best_alg} (Score: {score:.3f})")
        else:
            print(f"   • {env_name}: Aucun algorithme performant")
    
    # Performance globale des algorithmes
    alg_global_scores = {'MonteCarloES': [], 'OnPolicyMC': [], 'OffPolicyMC': []}
    
    for env_name, env_results in all_results.items():
        for alg_name, result in env_results.items():
            if 'history' in result and result['history'] and alg_name in alg_global_scores:
                final_rewards = [h['reward'] for h in result['history'][-20:]]
                avg_performance = np.mean(final_rewards) if final_rewards else 0
                alg_global_scores[alg_name].append(avg_performance)
    
    print("\n🌟 PERFORMANCE GLOBALE DES ALGORITHMES:")
    for alg_name, scores in alg_global_scores.items():
        if scores:
            avg_score = np.mean(scores)
            std_score = np.std(scores)
            print(f"   • {alg_name}: {avg_score:.3f} (±{std_score:.3f})")
        else:
            print(f"   • {alg_name}: Aucune donnée valide")
    
    # Recommandations spécifiques
    print("\n🎯 RECOMMANDATIONS SPÉCIFIQUES:")
    print("   1. 🔄 MonteCarloES excelle sur les environnements nécessitant une exploration intensive")
    print("   2. ⚖️  OnPolicyMC offre un bon équilibre exploration/exploitation")
    print("   3. 🎯 OffPolicyMC peut être instable mais performant sur certains environnements") 
    print("   4. 📊 Surveillez les courbes de convergence pour détecter l'instabilité")
    print("   5. 🎛️  Ajustez γ et ε selon les caractéristiques spécifiques de chaque environnement")
    
    print("\n💾 Pour sauvegarder les résultats, consultez le CSV généré automatiquement.")
    print("=" * 70)

print("📊 Fonctions d'analyse et de visualisation définies !")


In [ ]:
# 📈 Génération Complète des Résultats et Analyses

if all_results and any(env_results for env_results in all_results.values()):
    
    print("📈 GÉNÉRATION DES ANALYSES VISUELLES COMPLÈTES")
    print("=" * 70)
    
    # 1. 📊 Courbes d'apprentissage pour chaque environnement
    print("\n🎯 1. COURBES D'APPRENTISSAGE PAR ENVIRONNEMENT")
    print("-" * 60)
    
    for env_name, env_results in all_results.items():
        # Vérifier qu'on a au moins un résultat valide pour cet environnement
        has_valid_results = any('history' in result and result['history'] 
                               for result in env_results.values())
        
        if has_valid_results:
            print(f"\\n📊 Génération des graphiques pour {env_name}...")
            plot_learning_curves(env_results, title_prefix=env_name)
        else:
            print(f"❌ Pas de données valides pour {env_name}")
    
    # 2. 🌡️ Heatmap comparative des performances
    print("\\n🎯 2. HEATMAP COMPARATIVE DES PERFORMANCES")
    print("-" * 60)
    
    try:
        plot_performance_heatmap(all_results)
        print("✅ Heatmap générée avec succès")
    except Exception as e:
        print(f"❌ Erreur génération heatmap: {e}")
    
    # 3. 🔍 Analyse détaillée des résultats
    print("\\n🎯 3. ANALYSE DÉTAILLÉE DES RÉSULTATS")
    print("-" * 60)
    
    try:
        summary_df = analyze_results_detailed(all_results)
        print("✅ Analyse détaillée terminée")
    except Exception as e:
        print(f"❌ Erreur analyse détaillée: {e}")
        summary_df = pd.DataFrame()
    
    # 4. 💡 Recommandations et conclusions
    print("\\n🎯 4. RECOMMANDATIONS ET CONCLUSIONS")
    print("-" * 60)
    
    try:
        generate_recommendations(all_results)
        print("✅ Recommandations générées")
    except Exception as e:
        print(f"❌ Erreur génération recommandations: {e}")
    
    # 5. 💾 Sauvegarde des résultats
    print("\\n🎯 5. SAUVEGARDE DES RÉSULTATS")
    print("-" * 60)
    
    try:
        if not summary_df.empty:
            csv_filename = 'monte_carlo_secret_env_results.csv'
            summary_df.to_csv(csv_filename, index=False)
            print(f"✅ Résultats sauvegardés dans '{csv_filename}'")
            
            # Sauvegarder également les données complètes
            detailed_results = []
            for env_name, env_results in all_results.items():
                for alg_name, result in env_results.items():
                    if 'history' in result and result['history']:
                        for episode_data in result['history']:
                            row = {
                                'Environnement': env_name,
                                'Algorithme': alg_name,
                                **episode_data
                            }
                            detailed_results.append(row)
            
            if detailed_results:
                detailed_df = pd.DataFrame(detailed_results)
                detailed_csv = 'monte_carlo_detailed_history.csv'
                detailed_df.to_csv(detailed_csv, index=False)
                print(f"✅ Historique détaillé sauvegardé dans '{detailed_csv}'")
        else:
            print("❌ Aucune donnée à sauvegarder")
            
    except Exception as e:
        print(f"❌ Erreur sauvegarde: {e}")
    
    # 6. 📊 Résumé final avec métriques clés
    print("\\n🎯 6. RÉSUMÉ FINAL")
    print("=" * 70)
    
    total_combinations = 0
    successful_combinations = 0
    
    for env_name, env_results in all_results.items():
        for alg_name, result in env_results.items():
            total_combinations += 1
            if 'history' in result and result['history']:
                successful_combinations += 1
    
    success_percentage = (successful_combinations / total_combinations * 100) if total_combinations > 0 else 0
    
    print(f"📈 Combinaisons réussies: {successful_combinations}/{total_combinations} ({success_percentage:.1f}%)")
    print(f"🎮 Environnements testés: {len(all_results)}")
    print(f"🧠 Algorithmes utilisés: MonteCarloES, OnPolicyMC, OffPolicyMC")
    print(f"💾 Fichiers générés: CSV avec résultats et historiques détaillés")
    
    if successful_combinations > 0:
        print("\\n🎉 ANALYSE MONTE CARLO TERMINÉE AVEC SUCCÈS !")
        print("🕵️ Les algorithmes Monte Carlo ont révélé les secrets des environnements !")
        
        # Afficher quelques statistiques finales intéressantes
        best_overall_performance = -float('inf')
        best_combination = None
        
        for env_name, env_results in all_results.items():
            for alg_name, result in env_results.items():
                if 'history' in result and result['history']:
                    final_rewards = [h['reward'] for h in result['history'][-10:]]
                    avg_final_performance = np.mean(final_rewards)
                    
                    if avg_final_performance > best_overall_performance:
                        best_overall_performance = avg_final_performance
                        best_combination = (alg_name, env_name)
        
        if best_combination:
            print(f"🏆 Meilleure combinaison globale: {best_combination[0]} sur {best_combination[1]}")
            print(f"   Performance finale: {best_overall_performance:.3f}")
    else:
        print("⚠️ Analyse terminée mais aucun résultat valide obtenu")
        print("   Vérifiez la compatibilité des environnements secrets")

else:
    print("❌ AUCUN RÉSULTAT À ANALYSER")
    print("=" * 70)
    print("🔍 Vérifications à effectuer:")
    print("   1. Les environnements secrets sont-ils accessibles ?")
    print("   2. Les adaptateurs ont-ils été créés correctement ?") 
    print("   3. Les algorithmes Monte Carlo sont-ils importés ?")
    print("   4. L'entraînement s'est-il exécuté sans erreur ?")
    print("\\n💡 Conseil: Relancez les cellules précédentes pour diagnostiquer le problème")

print("\\n" + "=" * 70)
print("🔚 FIN DE L'ANALYSE MONTE CARLO SUR LES ENVIRONNEMENTS SECRETS")
print("=" * 70)


In [ ]:
# 📊 Fonctions de visualisation

def plot_learning_curves(results_dict, title_prefix=""):
    """Affiche les courbes d'apprentissage"""
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    axes = axes.flatten()
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
    
    # 1. Récompenses par épisode
    ax1 = axes[0]
    for i, (alg_name, result) in enumerate(results_dict.items()):
        if 'history' in result and result['history']:
            history = result['history']
            episodes = [h['episode'] for h in history]
            rewards = [h['reward'] for h in history]
            
            # Moyenne mobile
            if len(rewards) >= 20:
                rewards_smooth = pd.Series(rewards).rolling(window=20, min_periods=1).mean()
                ax1.plot(episodes, rewards_smooth, label=alg_name, color=colors[i], linewidth=2)
            else:
                ax1.plot(episodes, rewards, label=alg_name, color=colors[i], linewidth=2)
    
    ax1.set_title(f'{title_prefix} - Récompenses par Épisode')
    ax1.set_xlabel('Épisode')
    ax1.set_ylabel('Récompense')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Q-values moyennes
    ax2 = axes[1]
    for i, (alg_name, result) in enumerate(results_dict.items()):
        if 'history' in result and result['history']:
            history = result['history']
            episodes = [h['episode'] for h in history]
            avg_q = [h['avg_q'] for h in history]
            ax2.plot(episodes, avg_q, label=alg_name, color=colors[i], linewidth=2)
    
    ax2.set_title(f'{title_prefix} - Évolution des Q-values')
    ax2.set_xlabel('Épisode')
    ax2.set_ylabel('Q-value Moyenne')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. Longueur des épisodes
    ax3 = axes[2]
    for i, (alg_name, result) in enumerate(results_dict.items()):
        if 'history' in result and result['history']:
            history = result['history']
            episodes = [h['episode'] for h in history]
            lengths = [h['length'] for h in history]
            
            if len(lengths) >= 20:
                lengths_smooth = pd.Series(lengths).rolling(window=20, min_periods=1).mean()
                ax3.plot(episodes, lengths_smooth, label=alg_name, color=colors[i], linewidth=2)
            else:
                ax3.plot(episodes, lengths, label=alg_name, color=colors[i], linewidth=2)
    
    ax3.set_title(f'{title_prefix} - Longueur des Épisodes')
    ax3.set_xlabel('Épisode')
    ax3.set_ylabel('Nombre de Steps')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # 4. Taux de succès cumulé
    ax4 = axes[3]
    for i, (alg_name, result) in enumerate(results_dict.items()):
        if 'history' in result and result['history']:
            history = result['history']
            episodes = [h['episode'] for h in history]
            
            # Calculer taux de succès cumulé
            success_rates = []
            successes = 0
            for j, h in enumerate(history):
                if h['successful']:
                    successes += 1
                success_rates.append(successes / (j + 1))
            
            ax4.plot(episodes, success_rates, label=alg_name, color=colors[i], linewidth=2)
    
    ax4.set_title(f'{title_prefix} - Taux de Succès Cumulé')
    ax4.set_xlabel('Épisode')
    ax4.set_ylabel('Taux de Succès')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

def analyze_results(all_results):
    """Analyse détaillée des résultats"""
    print("🔍 ANALYSE DÉTAILLÉE DES RÉSULTATS")
    print("=" * 60)
    
    summary_data = []
    
    for env_name, env_results in all_results.items():
        print(f"\\n📊 {env_name.upper()}")
        print("-" * 40)
        
        for alg_name, result in env_results.items():
            if 'history' in result and result['history']:
                history = result['history']
                
                # Statistiques
                total_episodes = len(history)
                successful_episodes = sum(1 for h in history if h['successful'])
                success_rate = successful_episodes / total_episodes
                
                all_rewards = [h['reward'] for h in history]
                avg_reward = np.mean(all_rewards)
                std_reward = np.std(all_rewards)
                
                # Performance finale (derniers 20%)
                final_portion = history[int(0.8 * len(history)):]
                final_rewards = [h['reward'] for h in final_portion]
                final_avg_reward = np.mean(final_rewards) if final_rewards else 0
                
                print(f"\\n🎯 {alg_name}:")
                print(f"   • Taux de succès: {success_rate:.1%}")
                print(f"   • Récompense moyenne: {avg_reward:.3f} (±{std_reward:.3f})")
                print(f"   • Performance finale: {final_avg_reward:.3f}")
                
                summary_data.append({
                    'Environnement': env_name,
                    'Algorithme': alg_name,
                    'Taux_Succès': f"{success_rate:.1%}",
                    'Récompense_Moyenne': f"{avg_reward:.3f}",
                    'Récompense_Finale': f"{final_avg_reward:.3f}",
                    'Épisodes': total_episodes
                })
            else:
                print(f"\\n❌ {alg_name}: Aucune donnée")
                summary_data.append({
                    'Environnement': env_name,
                    'Algorithme': alg_name,
                    'Taux_Succès': "0%",
                    'Récompense_Moyenne': "0.000",
                    'Récompense_Finale': "0.000",
                    'Épisodes': 0
                })
    
    # Tableau récapitulatif
    summary_df = pd.DataFrame(summary_data)
    print("\\n📋 TABLEAU RÉCAPITULATIF:")
    print(summary_df.to_string(index=False))
    
    return summary_df

print("📊 Fonctions de visualisation définies !")


In [ ]:
# 🚀 Entraînement Principal

def run_monte_carlo_analysis(num_episodes=300):
    """Lance l'analyse complète"""
    
    print("🚀 DÉBUT DE L'ANALYSE MONTE CARLO")
    print("=" * 60)
    print(f"Paramètres: {num_episodes} épisodes par algorithme")
    print("=" * 60)
    
    all_results = {}
    
    for env_name, adapter in adapters.items():
        print(f"\\n🎮 ENVIRONNEMENT: {env_name}")
        print(f"États: {adapter.nS}, Actions: {adapter.nA}")
        print("-" * 50)
        
        env_results = {}
        
        # 1. Monte Carlo Exploring Starts
        print("\\n🎯 Entraînement Monte Carlo ES...")
        try:
            mc_es = SecretMonteCarloES(adapter, gamma=0.99, name=f"MC-ES-{env_name}")
            result_es = mc_es.train(num_episodes=num_episodes)
            env_results['MC-ES'] = result_es
            print(f"✅ MC-ES terminé - Succès: {result_es['success_rate']:.2%}")
        except Exception as e:
            print(f"❌ Erreur MC-ES: {e}")
            env_results['MC-ES'] = {'history': [], 'success_rate': 0}
        
        # 2. On-Policy Monte Carlo
        print("\\n🎯 Entraînement On-Policy MC...")
        try:
            on_policy_mc = SecretOnPolicyMC(adapter, gamma=0.99, epsilon=0.4, name=f"OnPolicy-{env_name}")
            result_on = on_policy_mc.train(num_episodes=num_episodes)
            env_results['On-Policy MC'] = result_on
            print(f"✅ On-Policy MC terminé - Succès: {result_on['success_rate']:.2%}")
        except Exception as e:
            print(f"❌ Erreur On-Policy MC: {e}")
            env_results['On-Policy MC'] = {'history': [], 'success_rate': 0}
        
        all_results[env_name] = env_results
        
        # Résumé pour cet environnement
        print(f"\\n📊 RÉSUMÉ {env_name}:")
        for alg_name, result in env_results.items():
            if result['history']:
                final_rewards = [h['reward'] for h in result['history'][-20:]]
                avg_final_reward = np.mean(final_rewards) if final_rewards else 0
                print(f"   • {alg_name}: Récompense finale = {avg_final_reward:.3f}")
            else:
                print(f"   • {alg_name}: ❌ Échec")
    
    print("\\n🎉 ANALYSE COMPLÈTE TERMINÉE !")
    return all_results

# Lancer l'analyse
if adapters:  # Seulement si les adaptateurs ont été créés
    print("⏳ Lancement de l'analyse (cela peut prendre 5-10 minutes)...")
    EPISODES = 300  # Ajustez selon vos besoins
    
    all_results = run_monte_carlo_analysis(num_episodes=EPISODES)
else:
    print("❌ Impossible de lancer l'analyse - adaptateurs non disponibles")
    all_results = {}


In [ ]:
# 📈 Affichage des Résultats

if all_results:
    print("📈 GÉNÉRATION DES ANALYSES VISUELLES")
    print("=" * 60)
    
    # 1. Courbes d'apprentissage pour chaque environnement
    print("\\n🎯 1. COURBES D'APPRENTISSAGE PAR ENVIRONNEMENT")
    
    for env_name, env_results in all_results.items():
        if any(result['history'] for result in env_results.values() if 'history' in result):
            print(f"\\n📊 Graphiques pour {env_name}...")
            plot_learning_curves(env_results, title_prefix=env_name)
        else:
            print(f"❌ Pas de données pour {env_name}")
    
    # 2. Analyse détaillée
    print("\\n🎯 2. ANALYSE DÉTAILLÉE")
    print("-" * 50)
    summary_df = analyze_results(all_results)
    
    # 3. Recommandations
    print("\\n🎯 3. RECOMMANDATIONS")
    print("=" * 60)
    
    best_performers = {}
    for env_name, env_results in all_results.items():
        best_alg = None
        best_score = -float('inf')
        
        for alg_name, result in env_results.items():
            if 'history' in result and result['history']:
                final_rewards = [h['reward'] for h in result['history'][-50:]]
                avg_reward = np.mean(final_rewards) if final_rewards else 0
                success_rate = result['success_rate']
                
                # Score composite
                composite_score = avg_reward * 0.7 + success_rate * 0.3
                
                if composite_score > best_score:
                    best_score = composite_score
                    best_alg = alg_name
        
        best_performers[env_name] = (best_alg, best_score)
    
    print("\\n🏆 MEILLEURS ALGORITHMES PAR ENVIRONNEMENT:")
    for env_name, (best_alg, score) in best_performers.items():
        if best_alg:
            print(f"   • {env_name}: {best_alg} (Score: {score:.3f})")
        else:
            print(f"   • {env_name}: Aucun algorithme efficace")
    
    # 4. Performance globale
    alg_scores = {'MC-ES': [], 'On-Policy MC': []}
    
    for env_name, env_results in all_results.items():
        for alg_name, result in env_results.items():
            if 'history' in result and result['history'] and alg_name in alg_scores:
                final_rewards = [h['reward'] for h in result['history'][-50:]]
                avg_reward = np.mean(final_rewards) if final_rewards else 0
                success_rate = result['success_rate']
                composite_score = avg_reward * 0.7 + success_rate * 0.3
                alg_scores[alg_name].append(composite_score)
    
    print("\\n🌟 PERFORMANCE GLOBALE:")
    for alg_name, scores in alg_scores.items():
        if scores:
            avg_score = np.mean(scores)
            std_score = np.std(scores)
            print(f"   • {alg_name}: {avg_score:.3f} (±{std_score:.3f})")
    
    # 5. Conseils
    print("\\n💡 CONSEILS D'INTERPRÉTATION:")
    print("   1. 🎯 Taux de succès élevé = algorithme stable")
    print("   2. 🔄 Récompenses croissantes = apprentissage effectif")
    print("   3. 📊 Q-values convergentes = politique stable")
    print("   4. 🎛️  Ajustez les hyperparamètres si nécessaire")
    
    # Sauvegarde
    try:
        summary_df.to_csv('monte_carlo_results.csv', index=False)
        print(f"\\n💾 Résultats sauvegardés dans 'monte_carlo_results.csv'")
    except Exception as e:
        print(f"❌ Erreur sauvegarde: {e}")
    
else:
    print("❌ AUCUN RÉSULTAT À AFFICHER")
    print("Vérifiez que l'entraînement précédent s'est bien déroulé.")

print("\\n🎉 ANALYSE MONTE CARLO TERMINÉE !")
print("🕵️ Les environnements secrets ont révélé leurs mystères !")
print("=" * 60)


In [ ]:
# 📚 Imports et configuration
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
from typing import Dict, List, Tuple, Any
import warnings
warnings.filterwarnings('ignore')

# Configuration matplotlib
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (14, 10)
plt.rcParams['font.size'] = 12
sns.set_palette("husl")

# Ajouter les chemins nécessaires
project_root = os.path.abspath('../../')
sys.path.insert(0, project_root)
sys.path.insert(0, os.path.join(project_root, 'game', 'secret_env'))

# Imports des environnements secrets
try:
    from secret_envs_wrapper import SecretEnv0, SecretEnv1, SecretEnv2, SecretEnv3
    print("✅ Environnements secrets importés avec succès")
except Exception as e:
    print(f"❌ Erreur d'import des environnements secrets: {e}")
    # Fallback pour les imports
    import ctypes
    import platform
    
print("🔧 Configuration terminée !")

# Test rapide des environnements
try:
    env0 = SecretEnv0()
    print(f"📊 SecretEnv0 - États: {env0.num_states()}, Actions: {env0.num_actions()}")
    
    env1 = SecretEnv1()
    print(f"📊 SecretEnv1 - États: {env1.num_states()}, Actions: {env1.num_actions()}")
    
    env2 = SecretEnv2()
    print(f"📊 SecretEnv2 - États: {env2.num_states()}, Actions: {env2.num_actions()}")
    
    env3 = SecretEnv3()
    print(f"📊 SecretEnv3 - États: {env3.num_states()}, Actions: {env3.num_actions()}")
    
    print("\n🎉 Tous les environnements secrets sont fonctionnels !")
    
except Exception as e:
    print(f"❌ Erreur lors du test des environnements: {e}")
    raise


In [ ]:
# 🔧 Adaptateur d'environnement pour les Secret Envs

class SecretEnvAdapter:
    """
    Adaptateur pour rendre les SecretEnv compatibles avec l'API Gym standard.
    Transforme l'interface spécifique des environnements secrets en interface standard.
    """
    
    def __init__(self, secret_env_class, env_name="SecretEnv"):
        self.secret_env_class = secret_env_class
        self.env_name = env_name
        self.env = secret_env_class()
        
        # Propriétés MDP pour compatibilité avec Monte Carlo
        self.nS = self.env.num_states()
        self.nA = self.env.num_actions()
        
        # État et récompenses
        self.current_state = None
        self.last_score = 0.0
        self.episode_steps = 0
        
        print(f"🏗️  {env_name} adapter créé - États: {self.nS}, Actions: {self.nA}")
    
    def reset(self):
        """Réinitialise l'environnement et retourne l'état initial"""
        try:
            self.env.reset()
            self.current_state = self.env.state_id()
            self.last_score = self.env.score()
            self.episode_steps = 0
            return self.current_state
        except Exception as e:
            print(f"❌ Erreur reset {self.env_name}: {e}")
            # Créer un nouvel environnement si reset échoue
            self.env = self.secret_env_class()
            self.env.reset()
            self.current_state = self.env.state_id()
            self.last_score = self.env.score()
            self.episode_steps = 0
            return self.current_state
    
    def step(self, action):
        """
        Exécute une action et retourne (next_state, reward, done, info)
        """
        try:
            # Obtenir les actions disponibles
            available_actions = self.get_available_actions()
            
            # Vérifier si l'action est valide
            if action not in available_actions:
                # Action non valide - retourner récompense négative et rester dans l'état
                return self.current_state, -0.1, False, {
                    'invalid_action': True,
                    'available_actions': available_actions,
                    'requested_action': action
                }
            
            # Sauvegarder le score avant l'action
            old_score = self.env.score()
            
            # Exécuter l'action
            self.env.step(action)
            self.episode_steps += 1
            
            # Obtenir le nouvel état et calculer la récompense
            next_state = self.env.state_id()
            new_score = self.env.score()
            reward = new_score - old_score  # Récompense différentielle
            done = self.env.is_game_over()
            
            # Mise à jour
            self.current_state = next_state
            self.last_score = new_score
            
            info = {
                'available_actions': self.get_available_actions(),
                'cumulative_score': new_score,
                'episode_steps': self.episode_steps,
                'valid_action': True
            }
            
            # Limite de sécurité pour éviter les épisodes infinis
            if self.episode_steps > 1000:
                done = True
                reward -= 1.0  # Pénalité pour épisode trop long
                info['timeout'] = True
            
            return next_state, reward, done, info
            
        except Exception as e:
            print(f"❌ Erreur step {self.env_name}: {e}")
            # Retourner un état d'erreur
            return self.current_state, -1.0, True, {'error': str(e)}
    
    def get_available_actions(self):
        """Obtient la liste des actions disponibles dans l'état courant"""
        try:
            actions = self.env.available_actions()
            return list(actions) if len(actions) > 0 else [0]
        except:
            # Fallback : toutes les actions sont disponibles
            return list(range(self.nA))
    
    def display(self):
        """Affiche l'état courant de l'environnement"""
        try:
            self.env.display()
        except:
            print(f"État courant: {self.current_state}, Score: {self.last_score}")
    
    def get_mdp_info(self):
        """Retourne les informations MDP pour compatibilité"""
        return {
            'states': list(range(self.nS)),
            'actions': list(range(self.nA)),
            'n_states': self.nS,
            'n_actions': self.nA,
            'name': self.env_name
        }

# Test des adaptateurs
print("🧪 Test des adaptateurs...")
adapters = {}

try:
    adapters['SecretEnv0'] = SecretEnvAdapter(SecretEnv0, "SecretEnv0")
    adapters['SecretEnv1'] = SecretEnvAdapter(SecretEnv1, "SecretEnv1")
    adapters['SecretEnv2'] = SecretEnvAdapter(SecretEnv2, "SecretEnv2")
    adapters['SecretEnv3'] = SecretEnvAdapter(SecretEnv3, "SecretEnv3")
    
    print("\n✅ Tous les adaptateurs créés avec succès !")
    
    # Test rapide d'un adaptateur
    test_adapter = adapters['SecretEnv0']
    state = test_adapter.reset()
    available = test_adapter.get_available_actions()
    print(f"🔍 Test SecretEnv0 - État initial: {state}, Actions disponibles: {available}")
    
except Exception as e:
    print(f"❌ Erreur lors de la création des adaptateurs: {e}")
    raise


In [ ]:
# 🎮 Implémentation des Algorithmes Monte Carlo pour les Environnements Secrets

class SecretMonteCarloES:
    """Monte Carlo avec Exploring Starts adapté aux environnements secrets"""
    
    def __init__(self, env_adapter, gamma=0.99, name="MC-ES"):
        self.env_adapter = env_adapter
        self.gamma = gamma
        self.name = name
        
        # Structures Monte Carlo
        self.nS = env_adapter.nS
        self.nA = env_adapter.nA
        self.Q = np.random.uniform(-0.1, 0.1, (self.nS, self.nA))  # Initialisation aléatoire
        self.policy = np.zeros(self.nS, dtype=int)
        self.returns_sum = defaultdict(float)
        self.returns_count = defaultdict(int)
        
        # Historique d'entraînement
        self.history = []
        
        print(f"🎯 {name} initialisé pour {env_adapter.env_name}")
    
    def generate_episode_with_exploring_starts(self):
        """Génère un épisode avec exploring starts"""
        episode = []
        
        # Reset avec état aléatoire (approximation d'exploring starts)
        for _ in range(10):  # Essayer plusieurs resets pour varier l'état initial
            state = self.env_adapter.reset()
            if np.random.random() < 0.3:  # 30% chance d'accepter cet état
                break
        
        # Action initiale aléatoire (exploring starts)
        available_actions = self.env_adapter.get_available_actions()
        if len(available_actions) > 0:
            action = np.random.choice(available_actions)
        else:
            action = 0
        
        done = False
        steps = 0
        max_steps = 500
        
        while not done and steps < max_steps:
            next_state, reward, done, info = self.env_adapter.step(action)
            episode.append((state, action, reward))
            
            if done:
                break
                
            # Action suivante selon politique courante avec actions disponibles
            state = next_state
            available_actions = info.get('available_actions', list(range(self.nA)))
            
            if len(available_actions) > 0:
                # Politique greedy avec tie-breaking aléatoire sur actions disponibles
                q_vals = np.array([self.Q[state, a] for a in available_actions])
                max_q = np.max(q_vals)
                best_actions = [a for a in available_actions if self.Q[state, a] == max_q]
                action = np.random.choice(best_actions)
            else:
                break
            
            steps += 1
        
        return episode
    
    def train(self, num_episodes=1000):
        """Entraînement Monte Carlo ES"""
        self.history = []
        successful_episodes = 0
        
        for episode_num in range(num_episodes):
            try:
                # Générer épisode avec exploring starts
                episode = self.generate_episode_with_exploring_starts()
                
                if len(episode) > 0:
                    successful_episodes += 1
                    
                    # Mise à jour First-Visit Monte Carlo
                    G = 0.0
                    visited = set()
                    
                    for (state, action, reward) in reversed(episode):
                        G = self.gamma * G + reward
                        
                        if (state, action) not in visited:
                            visited.add((state, action))
                            self.returns_count[(state, action)] += 1
                            self.returns_sum[(state, action)] += G
                            self.Q[state, action] = self.returns_sum[(state, action)] / self.returns_count[(state, action)]
                    
                    # Amélioration de la politique (greedy)
                    for s in range(self.nS):
                        self.policy[s] = np.argmax(self.Q[s])
                    
                    # Statistiques
                    episode_reward = sum(r for _, _, r in episode)
                    avg_q = np.mean(self.Q)
                    
                    self.history.append({
                        'episode': episode_num + 1,
                        'reward': episode_reward,
                        'length': len(episode),
                        'avg_q': avg_q,
                        'successful': True
                    })
                else:
                    # Épisode échoué
                    self.history.append({
                        'episode': episode_num + 1,
                        'reward': 0.0,
                        'length': 0,
                        'avg_q': np.mean(self.Q),
                        'successful': False
                    })
                
                if (episode_num + 1) % 200 == 0:
                    success_rate = successful_episodes / (episode_num + 1)
                    recent_rewards = [h['reward'] for h in self.history[-50:]]
                    avg_recent_reward = np.mean(recent_rewards) if recent_rewards else 0
                    print(f"[{self.name}] Épisode {episode_num + 1}: "
                          f"Taux de succès: {success_rate:.2f}, "
                          f"Récompense récente: {avg_recent_reward:.3f}")
                          
            except Exception as e:
                print(f"❌ Erreur épisode {episode_num + 1}: {e}")
                self.history.append({
                    'episode': episode_num + 1,
                    'reward': 0.0,
                    'length': 0,
                    'avg_q': np.mean(self.Q),
                    'successful': False
                })
        
        return {
            'Q': self.Q,
            'policy': self.policy,
            'history': self.history,
            'success_rate': successful_episodes / num_episodes
        }

class SecretOnPolicyMC:
    """On-Policy Monte Carlo avec ε-greedy adapté aux environnements secrets"""
    
    def __init__(self, env_adapter, gamma=0.99, epsilon=0.3, name="On-Policy MC"):
        self.env_adapter = env_adapter
        self.gamma = gamma
        self.epsilon = epsilon
        self.initial_epsilon = epsilon
        self.name = name
        
        # Structures Monte Carlo
        self.nS = env_adapter.nS
        self.nA = env_adapter.nA
        self.Q = np.random.uniform(-0.1, 0.1, (self.nS, self.nA))
        self.policy = np.zeros(self.nS, dtype=int)
        self.returns_sum = defaultdict(float)
        self.returns_count = defaultdict(int)
        
        # Historique
        self.history = []
        
        print(f"🎯 {name} initialisé pour {env_adapter.env_name} (ε={epsilon})")
    
    def epsilon_greedy_action(self, state, available_actions):
        """Sélectionne une action selon ε-greedy parmi les actions disponibles"""
        if len(available_actions) == 0:
            return 0
        
        if np.random.random() < self.epsilon:
            return np.random.choice(available_actions)
        else:
            # Greedy : meilleure action parmi les disponibles
            q_vals = np.array([self.Q[state, a] for a in available_actions])
            max_q = np.max(q_vals)
            best_actions = [a for a in available_actions if self.Q[state, a] == max_q]
            return np.random.choice(best_actions)
    
    def generate_episode(self):
        """Génère un épisode selon la politique ε-greedy"""
        episode = []
        state = self.env_adapter.reset()
        done = False
        steps = 0
        max_steps = 500
        
        while not done and steps < max_steps:
            available_actions = self.env_adapter.get_available_actions()
            action = self.epsilon_greedy_action(state, available_actions)
            
            next_state, reward, done, info = self.env_adapter.step(action)
            episode.append((state, action, reward))
            
            state = next_state
            steps += 1
        
        return episode
    
    def train(self, num_episodes=1000):
        """Entraînement On-Policy Monte Carlo"""
        self.history = []
        successful_episodes = 0
        
        for episode_num in range(num_episodes):
            try:
                episode = self.generate_episode()
                
                if len(episode) > 0:
                    successful_episodes += 1
                    
                    # Mise à jour First-Visit Monte Carlo
                    G = 0.0
                    visited = set()
                    
                    for (state, action, reward) in reversed(episode):
                        G = self.gamma * G + reward
                        
                        if (state, action) not in visited:
                            visited.add((state, action))
                            self.returns_count[(state, action)] += 1
                            self.returns_sum[(state, action)] += G
                            self.Q[state, action] = self.returns_sum[(state, action)] / self.returns_count[(state, action)]
                    
                    # Amélioration de politique
                    for s in range(self.nS):
                        self.policy[s] = np.argmax(self.Q[s])
                    
                    # Décroissance d'epsilon
                    self.epsilon = max(0.01, self.epsilon * 0.9995)
                    
                    # Statistiques
                    episode_reward = sum(r for _, _, r in episode)
                    avg_q = np.mean(self.Q)
                    
                    self.history.append({
                        'episode': episode_num + 1,
                        'reward': episode_reward,
                        'length': len(episode),
                        'avg_q': avg_q,
                        'epsilon': self.epsilon,
                        'successful': True
                    })
                else:
                    self.history.append({
                        'episode': episode_num + 1,
                        'reward': 0.0,
                        'length': 0,
                        'avg_q': np.mean(self.Q),
                        'epsilon': self.epsilon,
                        'successful': False
                    })
                
                if (episode_num + 1) % 200 == 0:
                    success_rate = successful_episodes / (episode_num + 1)
                    recent_rewards = [h['reward'] for h in self.history[-50:]]
                    avg_recent_reward = np.mean(recent_rewards) if recent_rewards else 0
                    print(f"[{self.name}] Épisode {episode_num + 1}: "
                          f"Taux de succès: {success_rate:.2f}, "
                          f"ε: {self.epsilon:.3f}, "
                          f"Récompense récente: {avg_recent_reward:.3f}")
                          
            except Exception as e:
                print(f"❌ Erreur épisode {episode_num + 1}: {e}")
                
        return {
            'Q': self.Q,
            'policy': self.policy,
            'history': self.history,
            'success_rate': successful_episodes / num_episodes
        }

class SecretOffPolicyMC:
    """Off-Policy Monte Carlo avec Importance Sampling adapté aux environnements secrets"""
    
    def __init__(self, env_adapter, gamma=0.99, epsilon=0.4, name="Off-Policy MC"):
        self.env_adapter = env_adapter
        self.gamma = gamma
        self.epsilon = epsilon
        self.name = name
        
        # Structures Monte Carlo
        self.nS = env_adapter.nS
        self.nA = env_adapter.nA
        self.Q = np.random.uniform(-0.1, 0.1, (self.nS, self.nA))
        self.target_policy = np.zeros(self.nS, dtype=int)
        self.C = np.zeros((self.nS, self.nA))  # Poids cumulatifs
        
        # Historique
        self.history = []
        
        print(f"🎯 {name} initialisé pour {env_adapter.env_name} (ε={epsilon})")
    
    def behavior_policy(self, state, available_actions):
        """Politique de comportement ε-greedy"""
        if len(available_actions) == 0:
            return 0
            
        if np.random.random() < self.epsilon:
            return np.random.choice(available_actions)
        else:
            q_vals = np.array([self.Q[state, a] for a in available_actions])
            max_q = np.max(q_vals)
            best_actions = [a for a in available_actions if self.Q[state, a] == max_q]
            return np.random.choice(best_actions)
    
    def generate_episode(self):
        """Génère un épisode selon la politique de comportement"""
        episode = []
        state = self.env_adapter.reset()
        done = False
        steps = 0
        max_steps = 500
        
        while not done and steps < max_steps:
            available_actions = self.env_adapter.get_available_actions()
            action = self.behavior_policy(state, available_actions)
            
            next_state, reward, done, info = self.env_adapter.step(action)
            episode.append((state, action, reward, available_actions.copy()))
            
            state = next_state
            steps += 1
        
        return episode
    
    def train(self, num_episodes=1000):
        """Entraînement Off-Policy Monte Carlo avec Importance Sampling"""
        self.history = []
        successful_episodes = 0
        
        for episode_num in range(num_episodes):
            try:
                episode = self.generate_episode()
                
                if len(episode) > 0:
                    successful_episodes += 1
                    
                    # Importance Sampling Update
                    G = 0.0
                    W = 1.0
                    
                    for i in range(len(episode) - 1, -1, -1):
                        state, action, reward, available_actions = episode[i]
                        G = self.gamma * G + reward
                        
                        # Mettre à jour C et Q
                        self.C[state, action] += W
                        if self.C[state, action] > 0:
                            self.Q[state, action] += (W / self.C[state, action]) * (G - self.Q[state, action])
                        
                        # Mettre à jour la politique cible (greedy)
                        self.target_policy[state] = np.argmax(self.Q[state])
                        
                        # Vérifier si l'action est celle de la politique cible
                        if action != self.target_policy[state]:
                            break
                        
                        # Calculer le ratio d'importance
                        # Probabilité politique cible (déterministe)
                        target_prob = 1.0
                        
                        # Probabilité politique de comportement
                        if len(available_actions) > 0:
                            if action == np.argmax([self.Q[state, a] for a in available_actions]):
                                behavior_prob = 1.0 - self.epsilon + self.epsilon / len(available_actions)
                            else:
                                behavior_prob = self.epsilon / len(available_actions)
                        else:
                            behavior_prob = 1.0
                        
                        if behavior_prob > 0:
                            W *= target_prob / behavior_prob
                        else:
                            break
                    
                    # Statistiques
                    episode_reward = sum(r for _, _, r, _ in episode)
                    avg_q = np.mean(self.Q)
                    
                    self.history.append({
                        'episode': episode_num + 1,
                        'reward': episode_reward,
                        'length': len(episode),
                        'avg_q': avg_q,
                        'avg_weight': np.mean(self.C[self.C > 0]) if np.any(self.C > 0) else 0,
                        'successful': True
                    })
                else:
                    self.history.append({
                        'episode': episode_num + 1,
                        'reward': 0.0,
                        'length': 0,
                        'avg_q': np.mean(self.Q),
                        'avg_weight': 0,
                        'successful': False
                    })
                
                if (episode_num + 1) % 200 == 0:
                    success_rate = successful_episodes / (episode_num + 1)
                    recent_rewards = [h['reward'] for h in self.history[-50:]]
                    avg_recent_reward = np.mean(recent_rewards) if recent_rewards else 0
                    print(f"[{self.name}] Épisode {episode_num + 1}: "
                          f"Taux de succès: {success_rate:.2f}, "
                          f"Récompense récente: {avg_recent_reward:.3f}")
                          
            except Exception as e:
                print(f"❌ Erreur épisode {episode_num + 1}: {e}")
                
        return {
            'Q': self.Q,
            'policy': self.target_policy,
            'history': self.history,
            'success_rate': successful_episodes / num_episodes
        }

print("🎯 Algorithmes Monte Carlo définis avec succès !")


In [ ]:
# 📊 Fonctions de visualisation et d'analyse

def plot_learning_curves(results_dict, title_prefix=""):
    """Affiche les courbes d'apprentissage pour tous les algorithmes"""
    
    n_algorithms = len(results_dict)
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    axes = axes.flatten()
    
    # Couleurs pour chaque algorithme
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
    
    # 1. Récompenses par épisode
    ax1 = axes[0]
    for i, (alg_name, result) in enumerate(results_dict.items()):
        history = result['history']
        episodes = [h['episode'] for h in history]
        rewards = [h['reward'] for h in history]
        
        # Moyenne mobile pour lisser les courbes
        window_size = min(50, len(rewards) // 10 + 1)
        if len(rewards) >= window_size:
            rewards_smooth = pd.Series(rewards).rolling(window=window_size, min_periods=1).mean()
            ax1.plot(episodes, rewards_smooth, label=alg_name, color=colors[i % len(colors)], linewidth=2)
        else:
            ax1.plot(episodes, rewards, label=alg_name, color=colors[i % len(colors)], linewidth=2)
    
    ax1.set_title(f'{title_prefix} - Récompenses par Épisode', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Épisode')
    ax1.set_ylabel('Récompense')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Q-values moyennes
    ax2 = axes[1]
    for i, (alg_name, result) in enumerate(results_dict.items()):
        history = result['history']
        episodes = [h['episode'] for h in history]
        avg_q = [h['avg_q'] for h in history]
        
        ax2.plot(episodes, avg_q, label=alg_name, color=colors[i % len(colors)], linewidth=2)
    
    ax2.set_title(f'{title_prefix} - Évolution des Q-values', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Épisode')
    ax2.set_ylabel('Q-value Moyenne')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. Longueur des épisodes
    ax3 = axes[2]
    for i, (alg_name, result) in enumerate(results_dict.items()):
        history = result['history']
        episodes = [h['episode'] for h in history]
        lengths = [h['length'] for h in history]
        
        # Moyenne mobile
        window_size = min(50, len(lengths) // 10 + 1)
        if len(lengths) >= window_size:
            lengths_smooth = pd.Series(lengths).rolling(window=window_size, min_periods=1).mean()
            ax3.plot(episodes, lengths_smooth, label=alg_name, color=colors[i % len(colors)], linewidth=2)
        else:
            ax3.plot(episodes, lengths, label=alg_name, color=colors[i % len(colors)], linewidth=2)
    
    ax3.set_title(f'{title_prefix} - Longueur des Épisodes', fontsize=14, fontweight='bold')
    ax3.set_xlabel('Épisode')
    ax3.set_ylabel('Nombre de Steps')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # 4. Analyse de convergence (écart-type des récompenses récentes)
    ax4 = axes[3]
    for i, (alg_name, result) in enumerate(results_dict.items()):
        history = result['history']
        episodes = []
        stds = []
        
        window_size = 100
        for j in range(window_size, len(history)):
            recent_rewards = [h['reward'] for h in history[j-window_size:j]]
            episodes.append(history[j]['episode'])
            stds.append(np.std(recent_rewards))
        
        if len(episodes) > 0:
            ax4.plot(episodes, stds, label=alg_name, color=colors[i % len(colors)], linewidth=2)
    
    ax4.set_title(f'{title_prefix} - Stabilité (Écart-type des récompenses)', fontsize=14, fontweight='bold')
    ax4.set_xlabel('Épisode')
    ax4.set_ylabel('Écart-type des récompenses')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

def plot_performance_comparison(all_results):
    """Compare les performances finales de tous les algorithmes sur tous les environnements"""
    
    # Préparer les données pour la visualisation
    env_names = list(all_results.keys())
    alg_names = list(list(all_results.values())[0].keys())
    
    # Metrics à analyser
    final_rewards = []
    success_rates = []
    avg_q_values = []
    
    for env_name in env_names:
        env_rewards = []
        env_success_rates = []
        env_avg_q = []
        
        for alg_name in alg_names:
            result = all_results[env_name][alg_name]
            
            # Récompense finale (moyenne des 100 derniers épisodes)
            history = result['history']
            if len(history) >= 100:
                final_reward = np.mean([h['reward'] for h in history[-100:]])
            else:
                final_reward = np.mean([h['reward'] for h in history]) if history else 0
            
            env_rewards.append(final_reward)
            env_success_rates.append(result.get('success_rate', 0))
            env_avg_q.append(result['history'][-1]['avg_q'] if result['history'] else 0)
        
        final_rewards.append(env_rewards)
        success_rates.append(env_success_rates)
        avg_q_values.append(env_avg_q)
    
    # Créer les graphiques
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Heatmap des récompenses finales
    ax1 = axes[0, 0]
    im1 = ax1.imshow(final_rewards, cmap='RdYlGn', aspect='auto')
    ax1.set_xticks(range(len(alg_names)))
    ax1.set_xticklabels(alg_names, rotation=45)
    ax1.set_yticks(range(len(env_names)))
    ax1.set_yticklabels(env_names)
    ax1.set_title('Récompenses Finales par Algorithme et Environnement')
    
    # Ajouter les valeurs dans les cellules
    for i in range(len(env_names)):
        for j in range(len(alg_names)):
            ax1.text(j, i, f'{final_rewards[i][j]:.2f}', ha='center', va='center')
    
    plt.colorbar(im1, ax=ax1)
    
    # 2. Graphique en barres des taux de succès
    ax2 = axes[0, 1]
    x = np.arange(len(env_names))
    width = 0.25
    
    for i, alg_name in enumerate(alg_names):
        success_data = [success_rates[j][i] for j in range(len(env_names))]
        ax2.bar(x + i * width, success_data, width, label=alg_name, alpha=0.8)
    
    ax2.set_xlabel('Environnements')
    ax2.set_ylabel('Taux de Succès')
    ax2.set_title('Taux de Succès par Environnement et Algorithme')
    ax2.set_xticks(x + width * 1.5)
    ax2.set_xticklabels(env_names)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. Comparaison des Q-values moyennes finales
    ax3 = axes[1, 0]
    for i, alg_name in enumerate(alg_names):
        q_data = [avg_q_values[j][i] for j in range(len(env_names))]
        ax3.bar(x + i * width, q_data, width, label=alg_name, alpha=0.8)
    
    ax3.set_xlabel('Environnements')
    ax3.set_ylabel('Q-value Moyenne Finale')
    ax3.set_title('Q-values Finales par Environnement et Algorithme')
    ax3.set_xticks(x + width * 1.5)
    ax3.set_xticklabels(env_names)
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # 4. Graphique radar des performances générales
    ax4 = axes[1, 1]
    
    # Normaliser les métriques pour le radar
    final_rewards_norm = np.array(final_rewards)
    success_rates_norm = np.array(success_rates)
    
    # Score composite pour chaque algorithme
    composite_scores = []
    for i, alg_name in enumerate(alg_names):
        alg_rewards = [final_rewards[j][i] for j in range(len(env_names))]
        alg_success = [success_rates[j][i] for j in range(len(env_names))]
        
        # Score composite (moyenne pondérée)
        composite_score = np.mean(alg_rewards) * 0.7 + np.mean(alg_success) * 0.3
        composite_scores.append(composite_score)
    
    bars = ax4.bar(alg_names, composite_scores, color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'][:len(alg_names)])
    ax4.set_title('Score Composite Global par Algorithme')
    ax4.set_ylabel('Score Composite')
    
    # Ajouter les valeurs sur les barres
    for bar, score in zip(bars, composite_scores):
        height = bar.get_height()
        ax4.text(bar.get_x() + bar.get_width()/2., height,
                f'{score:.3f}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()

def analyze_algorithm_characteristics(all_results):
    """Analyse les caractéristiques spécifiques de chaque algorithme"""
    
    print("🔍 ANALYSE DÉTAILLÉE DES ALGORITHMES")
    print("=" * 60)
    
    for env_name, env_results in all_results.items():
        print(f"\n📊 {env_name.upper()}")
        print("-" * 40)
        
        for alg_name, result in env_results.items():
            history = result['history']
            
            if len(history) > 0:
                # Statistiques générales
                total_episodes = len(history)
                successful_episodes = sum(1 for h in history if h['successful'])
                success_rate = successful_episodes / total_episodes
                
                # Récompenses
                all_rewards = [h['reward'] for h in history]
                avg_reward = np.mean(all_rewards)
                std_reward = np.std(all_rewards)
                
                # Convergence (derniers 20% d'épisodes)
                final_portion = history[int(0.8 * len(history)):]
                final_rewards = [h['reward'] for h in final_portion]
                final_avg_reward = np.mean(final_rewards) if final_rewards else 0
                
                # Stabilité (écart-type des derniers épisodes)
                final_stability = np.std(final_rewards) if len(final_rewards) > 1 else 0
                
                print(f"\n🎯 {alg_name}:")
                print(f"   • Taux de succès: {success_rate:.1%}")
                print(f"   • Récompense moyenne: {avg_reward:.3f} (±{std_reward:.3f})")
                print(f"   • Performance finale: {final_avg_reward:.3f}")
                print(f"   • Stabilité finale: {final_stability:.3f}")
                
                # Caractéristiques spécifiques à l'algorithme
                if 'epsilon' in history[0]:
                    initial_eps = history[0]['epsilon']
                    final_eps = history[-1]['epsilon']
                    print(f"   • Décroissance ε: {initial_eps:.3f} → {final_eps:.3f}")
                
                if 'avg_weight' in history[0]:
                    final_weight = history[-1]['avg_weight']
                    print(f"   • Poids moyen final: {final_weight:.3f}")
            else:
                print(f"\n❌ {alg_name}: Aucune donnée d'entraînement")

print("📊 Fonctions de visualisation définies !")


In [ ]:
# 🚀 Entraînement Principal - Tous les Algorithmes sur Tous les Environnements

def run_complete_analysis(num_episodes=1000):
    """Lance l'analyse complète de tous les algorithmes sur tous les environnements"""
    
    print("🚀 DÉBUT DE L'ANALYSE COMPLÈTE")
    print("=" * 60)
    print(f"Paramètres: {num_episodes} épisodes par algorithme")
    print(f"Total: {4} environnements × {3} algorithmes = {12} entraînements")
    print("=" * 60)
    
    # Dictionnaire pour stocker tous les résultats
    all_results = {}
    
    # Environnements à tester
    env_classes = {
        'SecretEnv0': SecretEnv0,
        'SecretEnv1': SecretEnv1, 
        'SecretEnv2': SecretEnv2,
        'SecretEnv3': SecretEnv3
    }
    
    # Créer les adaptateurs
    adapters_dict = {}
    for env_name, env_class in env_classes.items():
        try:
            adapters_dict[env_name] = SecretEnvAdapter(env_class, env_name)
        except Exception as e:
            print(f"❌ Erreur création adaptateur {env_name}: {e}")
            continue
    
    print(f"\n✅ {len(adapters_dict)} adaptateurs créés avec succès")
    
    # Entraînement pour chaque environnement
    for env_name, adapter in adapters_dict.items():
        print(f"\n🎮 ENVIRONNEMENT: {env_name}")
        print(f"États: {adapter.nS}, Actions: {adapter.nA}")
        print("-" * 50)
        
        env_results = {}
        
        # 1. Monte Carlo Exploring Starts
        print("\\n🎯 Entraînement Monte Carlo ES...")
        try:
            mc_es = SecretMonteCarloES(adapter, gamma=0.99, name=f"MC-ES-{env_name}")
            result_es = mc_es.train(num_episodes=num_episodes)
            env_results['MC-ES'] = result_es
            print(f"✅ MC-ES terminé - Taux de succès: {result_es['success_rate']:.2%}")
        except Exception as e:
            print(f"❌ Erreur MC-ES sur {env_name}: {e}")
            env_results['MC-ES'] = {'history': [], 'success_rate': 0}
        
        # 2. On-Policy Monte Carlo
        print("\\n🎯 Entraînement On-Policy MC...")
        try:
            on_policy_mc = SecretOnPolicyMC(adapter, gamma=0.99, epsilon=0.3, name=f"OnPolicy-{env_name}")
            result_on = on_policy_mc.train(num_episodes=num_episodes)
            env_results['On-Policy MC'] = result_on
            print(f"✅ On-Policy MC terminé - Taux de succès: {result_on['success_rate']:.2%}")
        except Exception as e:
            print(f"❌ Erreur On-Policy MC sur {env_name}: {e}")
            env_results['On-Policy MC'] = {'history': [], 'success_rate': 0}
        
        # 3. Off-Policy Monte Carlo
        print("\\n🎯 Entraînement Off-Policy MC...")
        try:
            off_policy_mc = SecretOffPolicyMC(adapter, gamma=0.99, epsilon=0.4, name=f"OffPolicy-{env_name}")
            result_off = off_policy_mc.train(num_episodes=num_episodes)
            env_results['Off-Policy MC'] = result_off
            print(f"✅ Off-Policy MC terminé - Taux de succès: {result_off['success_rate']:.2%}")
        except Exception as e:
            print(f"❌ Erreur Off-Policy MC sur {env_name}: {e}")
            env_results['Off-Policy MC'] = {'history': [], 'success_rate': 0}
        
        # Stocker les résultats de cet environnement
        all_results[env_name] = env_results
        
        # Afficher un résumé pour cet environnement
        print(f"\\n📊 RÉSUMÉ {env_name}:")
        for alg_name, result in env_results.items():
            if result['history']:
                final_rewards = [h['reward'] for h in result['history'][-50:]]
                avg_final_reward = np.mean(final_rewards) if final_rewards else 0
                print(f"   • {alg_name}: Récompense finale = {avg_final_reward:.3f}")
            else:
                print(f"   • {alg_name}: ❌ Aucun résultat")
    
    print("\\n🎉 ANALYSE COMPLÈTE TERMINÉE !")
    print("=" * 60)
    
    return all_results

# Lancer l'analyse complète (peut prendre plusieurs minutes)
print("⏳ Lancement de l'analyse complète...")
print("Cela peut prendre plusieurs minutes selon la complexité des environnements...")

# Utiliser un nombre d'épisodes raisonnable pour le test
EPISODES = 800  # Ajustez selon vos besoins de temps

all_results = run_complete_analysis(num_episodes=EPISODES)


In [ ]:
# 📈 Affichage des Résultats et Analyses Complètes

print("📈 GÉNÉRATION DES ANALYSES VISUELLES")
print("=" * 60)

# Vérifier qu'on a des résultats
if all_results and any(env_results for env_results in all_results.values()):
    
    # 1. Afficher les courbes d'apprentissage pour chaque environnement
    print("\\n🎯 1. COURBES D'APPRENTISSAGE PAR ENVIRONNEMENT")
    print("-" * 50)
    
    for env_name, env_results in all_results.items():
        if any(result['history'] for result in env_results.values()):
            print(f"\\n📊 Graphiques pour {env_name}...")
            plot_learning_curves(env_results, title_prefix=env_name)
        else:
            print(f"❌ Pas de données valides pour {env_name}")
    
    # 2. Comparaison des performances entre environnements
    print("\\n🎯 2. COMPARAISON GLOBALE DES PERFORMANCES")
    print("-" * 50)
    plot_performance_comparison(all_results)
    
    # 3. Analyse détaillée des caractéristiques
    print("\\n🎯 3. ANALYSE DÉTAILLÉE")
    print("-" * 50)
    analyze_algorithm_characteristics(all_results)
    
    # 4. Tableau récapitulatif final
    print("\\n🎯 4. TABLEAU RÉCAPITULATIF FINAL")
    print("=" * 60)
    
    # Créer un DataFrame pour le résumé
    summary_data = []
    
    for env_name, env_results in all_results.items():
        for alg_name, result in env_results.items():
            if result['history']:
                # Calculer les métriques finales
                history = result['history']
                final_rewards = [h['reward'] for h in history[-100:]] if len(history) >= 100 else [h['reward'] for h in history]
                
                summary_data.append({
                    'Environnement': env_name,
                    'Algorithme': alg_name,
                    'Taux_Succès': f"{result['success_rate']:.1%}",
                    'Récompense_Finale': f"{np.mean(final_rewards):.3f}",
                    'Stabilité': f"{np.std(final_rewards):.3f}",
                    'Épisodes_Total': len(history),
                    'Q_Moyenne_Finale': f"{history[-1]['avg_q']:.3f}" if history else "0.000"
                })
            else:
                summary_data.append({
                    'Environnement': env_name,
                    'Algorithme': alg_name,
                    'Taux_Succès': "0.0%",
                    'Récompense_Finale': "0.000",
                    'Stabilité': "N/A",
                    'Épisodes_Total': 0,
                    'Q_Moyenne_Finale': "0.000"
                })
    
    # Afficher le tableau
    summary_df = pd.DataFrame(summary_data)
    print("\\n📋 Résultats par Algorithme et Environnement:")
    print(summary_df.to_string(index=False))
    
    # 5. Recommandations finales
    print("\\n🎯 5. RECOMMANDATIONS ET CONCLUSIONS")
    print("=" * 60)
    
    # Trouver les meilleurs algorithmes par environnement
    best_performers = {}
    for env_name, env_results in all_results.items():
        best_alg = None
        best_score = -float('inf')
        
        for alg_name, result in env_results.items():
            if result['history']:
                # Score composite basé sur récompense finale et taux de succès
                final_rewards = [h['reward'] for h in result['history'][-100:]] if len(result['history']) >= 100 else [h['reward'] for h in result['history']]
                avg_reward = np.mean(final_rewards) if final_rewards else 0
                success_rate = result['success_rate']
                
                composite_score = avg_reward * 0.7 + success_rate * 0.3
                
                if composite_score > best_score:
                    best_score = composite_score
                    best_alg = alg_name
        
        best_performers[env_name] = (best_alg, best_score)
    
    print("\\n🏆 MEILLEURS ALGORITHMES PAR ENVIRONNEMENT:")
    for env_name, (best_alg, score) in best_performers.items():
        if best_alg:
            print(f"   • {env_name}: {best_alg} (Score: {score:.3f})")
        else:
            print(f"   • {env_name}: Aucun algorithme efficace")
    
    # Analyse globale
    alg_global_scores = {'MC-ES': [], 'On-Policy MC': [], 'Off-Policy MC': []}
    
    for env_name, env_results in all_results.items():
        for alg_name, result in env_results.items():
            if result['history'] and alg_name in alg_global_scores:
                final_rewards = [h['reward'] for h in result['history'][-100:]] if len(result['history']) >= 100 else [h['reward'] for h in result['history']]
                avg_reward = np.mean(final_rewards) if final_rewards else 0
                success_rate = result['success_rate']
                composite_score = avg_reward * 0.7 + success_rate * 0.3
                alg_global_scores[alg_name].append(composite_score)
    
    print("\\n🌟 PERFORMANCE GLOBALE DES ALGORITHMES:")
    for alg_name, scores in alg_global_scores.items():
        if scores:
            avg_score = np.mean(scores)
            std_score = np.std(scores)
            print(f"   • {alg_name}: {avg_score:.3f} (±{std_score:.3f})")
        else:
            print(f"   • {alg_name}: Aucune donnée valide")
    
    # Recommandations spécifiques
    print("\\n💡 RECOMMANDATIONS:")
    print("   1. 🎯 Chaque environnement secret semble avoir des caractéristiques uniques")
    print("   2. 🔄 L'exploration est cruciale - MC-ES peut être avantagé")
    print("   3. 📊 Surveillez les taux de succès autant que les récompenses")
    print("   4. ⚖️  L'importance sampling (Off-Policy) peut être instable sur certains environnements")
    print("   5. 🎛️  L'ajustement des hyperparamètres (ε, γ) est critique")
    
    # Sauvegarde des résultats
    try:
        summary_df.to_csv('secret_env_monte_carlo_results.csv', index=False)
        print(f"\\n💾 Résultats sauvegardés dans 'secret_env_monte_carlo_results.csv'")
    except Exception as e:
        print(f"❌ Erreur lors de la sauvegarde: {e}")
    
else:
    print("❌ AUCUN RÉSULTAT VALIDE TROUVÉ")
    print("Vérifiez que les environnements secrets sont accessibles et fonctionnels.")

print("\\n🎉 ANALYSE MONTE CARLO TERMINÉE !")
print("🕵️ Les mystères des environnements secrets ont été explorés par Monte Carlo !")
print("=" * 60)
